# Calculus

Scipy contains many features to help perform calculus operations.

## Differentiation

The ```scipy.misc.derivative``` [function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.misc.derivative.html) calculates the differential of a function at a given location by sampling values of the function around the location for which the derivative is requested.

<center><img src='https://raw.githubusercontent.com/coolernato/Numerical-Computing-in-Python-with-NumPy-and-SciPy/master/differential.png' />
<figcaption>Differentiation diagram</figcaption></center>
</figure>

For instance, to estimate the first derivative, the calculated result will be:

$ \frac{\textrm{d}y}{\textrm{d}x} =\frac{y(x+dx)-y(x-dx)}{2dx}$

When the ```scipy.misc.derivative``` function is called, the first argument is the function that is to be differentiated (which must accept a single argument and return a scalar value). The second is the value of the independent variable which the derivative is to be calculated at. It has a number of optional arguments which may be provided to describe how it should behave. ```order``` defines the number of points around the specified value of $x$ (this defaults to 3 if it is not specified and must be odd). ```dx``` specifies the spacing of these points (this defaults to 1). ```n``` specifies the number of times the function is to be differentiated before the differential is returned (this defaults to 1). The value of ```order``` must be greater than or equal to the value of ```n```.

Consider the function:

$f(x) = x^{3} + 3x^{2} - 2x +1$

We may calculate the first, second and third derivatives:

$f'(x) = 3x^{2} + 6x -2\\
f''(x) = 6x + 6\\
f'''(x) = 6$

And we can evaluate these at specific values of x. For instance:

$f'(1) = 7\\
f''(2) = 18\\
f'''(0) = 6$

We can calculate these values using SciPy.

In [ ]:
from scipy.misc import derivative

def sample_function(x):
  return(x ** 3 + 3 * x ** 2 - 2 * x + 1)

# Note that dx defaults to 1, n defaults to 1 and order defaults to 3
print("First order: ", derivative(sample_function, 1, dx=1e-3))
print("Second order: ", derivative(sample_function, 2, n=2))
print("Third order: ", derivative(sample_function, 0, n=3, order = 5))

The results returned by this functions are sensitive to the value of ```dx```. Choosing too small a value leads to a round-off error which can invalidate the entire result. Choosing too large a value can lead to errors as sampling values far from the central value causes a resolution error. This means it may require some experimentation to ensure the correct value is returned for the case in question. In general, higher derivatives require a larger value of ```dx```.

In [ ]:
from scipy.misc import derivative

# Some examples when values of dx produce the wrong answer
print("First order: ", derivative(sample_function, 1))
print("Third order: ", derivative(sample_function, 2, n=3, order = 5, dx=1e-8))

In addition, the function may produce incorrect results or raise an exception at values where the function is discontinuous, undefined or infinite:

In [ ]:
import math
from scipy.misc import derivative

# Differentiating at an inifinte value:
def one_over_x_squared(x):
  return(1 / x ** 2)

print(derivative(one_over_x_squared, 0))

In [ ]:
from scipy.misc import derivative

# Differentiating a discountinuous function
def step(x):
  if x>0:
    return 1
  else:
    return 0

print("dx of 1: ", derivative(step, 0))
print("dx of 1e-8: ", derivative(step, 0, dx=1e-8))

In [ ]:
import math
from scipy.misc import derivative

# Differentiating an undefined function:
print(derivative(math.log, -1))

All of these caveats means the function (like many in Numpy and Scipy) cannot be used indiscriminately and without thought. So make sure you consider the case of cases the function will be used for in your application before applying it.

## Integration

Scipy contains a [suite of tools](https://docs.scipy.org/doc/scipy/reference/integrate.html) which can be used to integrate functions. Let's look at the ```scipy.integrate.quad``` [function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html#scipy.integrate.quad). Again, we'll use the function:

$f(x) = x^{3} + 3x^{2} - 2x +1$

When we integrate this, we obtain:

$\int f(x)\textrm{d}x = \frac{x^{4}}{4} + x^{3} - x^{2} + x$

We can evaluate this integral over a fixed range and find:

$\int\limits_{0}^{1} f(x)\textrm{d}x = \frac{5}{4}$

We can call ```scipy.integrate.quad``` to evaluate this value. The first argument is the function (which must accept a single argument and return a scalar value). The second and third values are the upper and lower bounds of the definite integral.

In [ ]:
from scipy.integrate import quad

def sample_function(x):
  return(x ** 3 + 3 * x ** 2 - 2 * x + 1)

print(quad(sample_function, 0, 1))


Here, ```quad``` has returned a tuple containing two values. The first of these is the value of integral as evaluated by quad, the second is the estimate of the maximum error of the function.

## Exercise

For the following functions:

$ f(x) = \sin(x),\\g(x) = x$

calculate the following (correct values are given for you to check your results):

$
f'(1) \approx 0.54\\
g''(\pi) = 0\\
\int_{0}^{1}g(x)\textrm{d}x = 0.5\\
\int_{0}^{\pi}\left(f(x)+g(x)\right)\textrm{d}x \approx 6.93
$

For all cases, compare the value you calculate to the correct answers:

In [ ]:
#@title

import math
from scipy.misc import derivative
from scipy.integrate import quad

def f(x):
  return math.sin(x)

def g(x):
  return(x)

print("f'(1): ", derivative(f, 1, dx=1e-5))
print("g''(pi): ", derivative(g, math.pi, n=2))
# Note selecting the first element of the tuple to get just the answer
print("int g from 0 to 1: ", quad(g, 0, 1)[0])
# Add together the (first element of the) results from two calls to quad
print("int f+g from 0 to pi: ", quad(f, 0, math.pi)[0] + quad(g, 0, math.pi)[0])
# You could also have made a new function to return xsin(x) and called quad once

## Additional Arguments

Both ```scipy.misc.derivative``` and ```scipy.integrate.quad``` allow for functions which accept multiple arguments. The differentiation and integration will be performed with respect to the first argument passed to the function being operated on. The values of other functions may be specified by writing ```args=``` and then a series of values in parentheses separated by commas.

For example, in the cell below we differentiate and integrate a function which represents a quadratic polynomial.

In [ ]:
from scipy.misc import derivative
from scipy.integrate import quad

# Define a function to return a result of the form ax^2 + bx + c
def quadratic(x, a, b, c):
  return(a * x ** 2 + b * x + c)

# Differentiate the function with parameters a=1, b=2, c=3 at x=1
print("Differential:", derivative(quadratic, 1, args=(1,2,3), dx=1e-6))

# Integrate the function with parameters a=3, b=-2, c=5 between x=0 and x=1
print("Integral: ", quad(quadratic, 0, 1, args=(3,-2,5)))